In [1]:
import sys
sys.path.append('./../')
import json
import numpy as np
from skimage import color
from skimage import filters
import util

In [2]:
REQUEST = json.dumps({
'path' : {},
'args' : {}
})

# tellusAPIの画像を取得するだけのAPI

In [3]:
# GET /img/:img_type/:z/:x/:y

# 読み取り
request = json.loads(REQUEST)

# 必要項目抽出
img_type = request['path'].get('img_type')
x = request['path'].get('x')
y = request['path'].get('y')
z = request['path'].get('z')

if util.input_validation_check(x, y, z, img_type):
    img_np = util.get_image_using_tellus_api(x, y, z, img_type)
    if len(img_np) != 0:
        img_base64 = util.make_base64_image(img_np)
        print('<html><img src="data:image/png;base64, {}" /></html>'.format(img_base64))
    else:
        pass

ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`
ex. http://localhost:8889/img/osm/13/7252/3234


In [4]:
# ResponseInfo GET /img/:img_type/:z/:x/:y
print(json.dumps({"headers" : {"Content-Type" : "text/html"},"status" : 201}))

{"headers": {"Content-Type": "text/html"}, "status": 201}


# 光学画像を元にNDSI画像を返すAPI

In [5]:
def make_snow_filter(img_true):
    """明度を利用したフィルターを作成"""

    # RGB画像からHSV画像への変換
    img_hsv = color.rgb2hsv(img_true.astype("uint8"))
    img_v = img_hsv[:,:,2]

    # 0で初期化
    height, width= img_v.shape
    snow_filter = np.zeros((height, width)) 

    # 閾値にもとづいて2値化
    snow_filter = img_v < 70 / 255
    
    # RGBに変換
    snow_filter = color.gray2rgb(snow_filter)
    
    return snow_filter

In [6]:
def ditect_ndsi(img_band2, img_band4):
    """NDSI（Normalized Difference Snow Index）画像を作成"""
    
    # NDSIを計算
    img_NDSI =  (img_band2[:,:,1] - img_band4[:,:,1]) / (img_band2[:,:,1] + img_band4[:,:,1])
    
    #最大と最小を設定し、0~1に正規化
    img_NDSI = np.clip(img_NDSI + 0.3, 0, 1)
    
    # grayからRGBに変換
    height, width= img_NDSI.shape
    img_NDSI_rgb = np.zeros((height, width, 3))
    img_NDSI_rgb[:,:,0] = 0
    img_NDSI_rgb[:,:,1] = img_NDSI * 255 
    img_NDSI_rgb[:,:,2] = 255
    
    return img_NDSI_rgb

In [7]:
# GET /ndsi_img/:z/:x/:y

# 読み取り
request = json.loads(REQUEST)

# 必要項目抽出
x = request['path'].get('x')
y = request['path'].get('y')
z = request['path'].get('z')
# z="13"
# x="7252"
# y="3234"

if util.input_validation_check(x, y, z):

    #データ取得
    img_band1 = util.get_image_using_tellus_api(x, y, z, "band1")
    img_band2 = util.get_image_using_tellus_api(x, y, z, "band2")
    img_band3 = util.get_image_using_tellus_api(x, y, z, "band3")
    img_band4 = util.get_image_using_tellus_api(x, y, z, "band4")
    img_true = np.c_[img_band3[:,:,0:1], img_band2[:,:,1:2], img_band1[:,:,2:3]]
    
    #フィルター作成
    snow_filter = make_snow_filter(img_true)
    
    #NDSI画像取得
    img_ndsi = ditect_ndsi(img_band2, img_band4)
    
    #フィルター適用
    img_output = img_true * snow_filter + img_ndsi * (1- snow_filter)
    
    # 表示のために型変換
    img_output_base64 = util.make_base64_image(img_output.astype("uint8"))

    # 出力
    print('<html><img src="data:image/png;base64, {}" /></html>'.format(img_output_base64))


ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`
ex. http://localhost:8889/img/osm/13/7252/3234


In [8]:
# ResponseInfo GET  /ndsi_img/:z/:x/:y
print(json.dumps({"headers" : {"Content-Type" : "text/html"},"status" : 201}))

{"headers": {"Content-Type": "text/html"}, "status": 201}


# SAR画像を元に解析結果を返すAPI

In [9]:
def analysis_sar_diff(img_fuji_sep, img_fuji_dec):
    #差分を抽出
    img_fuji_diff =  img_fuji_sep - img_fuji_dec

    #ノイズ除去
    img_fuji_diff_gaus = filters.gaussian(img_fuji_diff, sigma=8)

    #正規化
    img_fuji_diff_gaus_norm = np.clip( img_fuji_diff_gaus * 10 ** 20 , 0, 255)

    # grayからRGBに変換
    height, width= img_fuji_diff_gaus_norm.shape
    img_output = np.zeros((height, width, 3))
    img_output[:,:,0] = 127
    img_output[:,:,1] = img_fuji_diff_gaus_norm
    img_output[:,:,2] = 255
    
    return img_output

In [10]:
# GET /sar_analysis_img

# 読み取り
request = json.loads(REQUEST)

img_fuji_sep, img_fuji_dec = util.get_local_sar_image()

img_output = analysis_sar_diff(img_fuji_sep, img_fuji_dec)

# 表示のために型変換
img_output_base64 = util.make_base64_image(img_output.astype("uint8"))

# 出力
print('<html><img src="data:image/png;base64, {}" /></html>'.format(img_output_base64))

/opt/conda/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from int64 to float64
  .format(dtypeobj_in, dtypeobj_out))
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# ResponseInfo GET  /sar_analysis_img
print(json.dumps({"headers" : {"Content-Type" : "text/html"},"status" : 201}))

{"headers": {"Content-Type": "text/html"}, "status": 201}
